[![Open CoT Correction In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/docto-rin/JMLE-LLM-Toolkit/blob/main/colab_notebooks/JMLE-Gemini-2.5-Pro-CoT-Dataset-Correction.ipynb)

## Init

In [ ]:
!pip install datasets

In [ ]:
use_secret = True

if use_secret:
    from google.colab import userdata
    from huggingface_hub import login as hf_login
    hf_login(userdata.get('HF_TOKEN'))
else:
    from huggingface_hub import notebook_login
    notebook_login()

from huggingface_hub import HfApi, HfFolder
from pprint import pprint

token = HfFolder.get_token()
api = HfApi()
user_info = api.whoami(token=token)
pprint(user_info)

## View CoT Dataset

In [9]:
from datasets import load_dataset
from collections import Counter

def display_dataset_summary(dataset_name, sample_size=3):
    dataset = load_dataset(dataset_name)

    # Dataset Structure Overview
    print("Dataset Structure:")
    print(dataset)

    # Information about each split
    for split in dataset.keys():
        print(f"\nSplit: {split}")
        split_data = dataset[split]
        print(f"Number of examples: {len(split_data)}")

        # Column names and their types
        print("\nColumns:")
        for column, feature in split_data.features.items():
            print(f"  - {column}: {feature}")

        # Sample rows
        print(f"\nSample data (first {sample_size} rows):")
        for i in range(min(sample_size, len(split_data))):
            print(f"\nSample {i+1}:")
            for key, value in split_data[i].items():
                print(f"  {key}: {value}")

    # Dataset statistics
    print("\nDataset Statistics:")
    for split in dataset.keys():
        print(f"\n{split} split:")
        split_data = dataset[split]

        # Text columns statistics
        for column in split_data.features:
            if split_data.features[column].dtype == 'string':
                lengths = [len(ex[column]) for ex in split_data if ex[column] is not None]
                if lengths:
                    avg_length = sum(lengths) / len(lengths)
                    max_length = max(lengths)
                    min_length = min(lengths)
                    print(f"  {column} length - Avg: {avg_length:.1f}, Min: {min_length}, Max: {max_length}")

        # Categorical columns distribution
        for column, feature in split_data.features.items():
            if hasattr(feature, 'names'):
                values = [ex[column] for ex in split_data]
                value_counts = Counter(values)
                print(f"  {column} distribution:")
                for value, count in value_counts.most_common():
                    percentage = count / len(values) * 100
                    print(f"    - {value}: {count} ({percentage:.1f}%)")
    return dataset

In [13]:
dataset = display_dataset_summary("doctorin/JMLE-CoT-gemini-2.5-pro-dataset")

Dataset Structure:
DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'choices', 'answer', 'cot', 'explanation', 'generation_info'],
        num_rows: 3356
    })
    unmatched: Dataset({
        features: ['id', 'question', 'choices', 'answer', 'cot', 'explanation', 'generation_info'],
        num_rows: 34
    })
})

Split: train
Number of examples: 3356

Columns:
  - id: Value(dtype='string', id=None)
  - question: Value(dtype='string', id=None)
  - choices: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
  - answer: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
  - cot: Value(dtype='string', id=None)
  - explanation: Value(dtype='string', id=None)
  - generation_info: Value(dtype='string', id=None)

Sample data (first 3 rows):

Sample 1:
  id: 117A01
  question: 母体背景と胎児疾患の組合せで正しいのはどれか。3 つ選べ。 
  choices: ['a. 高齢妊娠 ― 13trisomy', 'b. 風疹感染 ― 先天性心疾患', 'c. 妊娠高血圧症候群 ― 不整脈', 'd. 全身性エリテマトーデス ― 頭蓋内出血', 'e. パルボウイルス B19 感染 ― 貧血']


## Fix Manually

In [ ]:
!pip install -q ipywidgets # Colabに通常入っていますが念のため

import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import ast
from datasets import load_dataset, Dataset, DatasetDict
import json
import time # 保存後の遷移前に少し待つために使用（オプション）

# データセットのロード (既にロード済みなら不要)
try:
    # dataset 変数が既に存在するか確認
    _ = dataset['unmatched']
    print("既存の 'dataset' 変数を使用します。")
except NameError:
    print("データセットをロードします...")
    dataset = load_dataset("doctorin/JMLE-CoT-gemini-2.5-pro-dataset")
    print("ロード完了。")

unmatched_data = dataset['unmatched']
num_unmatched = len(unmatched_data)

# --- グローバル変数 ---
current_index = 0
# 修正後のデータをIDをキーとして格納する辞書
corrected_data_dict = {}
# 現在表示中のデータのIDを保持
current_data_id = None

# --- ウィジェットの定義 ---
# データ表示エリア
output_display = widgets.Output(layout={'border': '1px solid black', 'padding': '10px'})
# 編集・操作エリア
output_edit = widgets.Output(layout={'padding': '10px'})

# ナビゲーション (下部に配置)
prev_button = widgets.Button(description="<< 前へ", disabled=True)
next_button = widgets.Button(description="次へ >>", disabled=(num_unmatched <= 1))
index_label = widgets.HTML(value=f"データ: {current_index + 1} / {num_unmatched}")
progress_bar = widgets.FloatProgress(value=0, min=0, max=num_unmatched, description='進捗:', bar_style='info')
nav_bar = widgets.HBox([prev_button, index_label, next_button], layout={'justify_content': 'center', 'margin': '10px 0'})

# 編集用ウィジェット (output_edit 内で動的に生成)
answer_input = widgets.Text(description="Answer (リスト形式):", layout={'width': '95%'})
cot_input = widgets.Textarea(description="CoT:", layout={'height': '200px', 'width': '95%'})
explanation_input = widgets.Textarea(description="Explanation:", layout={'height': '100px', 'width': '95%'})
save_button = widgets.Button(description="修正を保存して次へ", button_style='success') # ボタン名を変更
skip_button = widgets.Button(description="スキップして次へ", button_style='warning')
status_label = widgets.HTML(value="") # 修正状態表示用

# 全データリスト表示用
list_output = widgets.Output(layout={'max_height': '300px', 'overflow_y': 'auto', 'border': '1px solid #ccc', 'padding': '5px'})
accordion = widgets.Accordion(children=[list_output])
accordion.set_title(0, '全データリスト (クリックで展開/更新)')
accordion.selected_index = None # 初期状態では閉じておく

# エクスポートボタン
export_button = widgets.Button(description="修正済みデータをJSONLでエクスポート", button_style='info')
export_output = widgets.Output()

# --- 関数定義 ---
def parse_generation_info(gen_info_str):
    """generation_info文字列を辞書にパース"""
    try:
        gen_info = ast.literal_eval(gen_info_str)
        if not isinstance(gen_info, dict):
            return {'error': 'パース結果が辞書ではありません'}
        return gen_info
    except (ValueError, SyntaxError, TypeError) as e:
        return {'error': f'パース失敗: {e}', 'raw': gen_info_str}

def display_data(index):
    """指定されたインデックスのデータを表示・編集UIにセット"""
    global current_data_id
    if not (0 <= index < num_unmatched):
        print(f"エラー: インデックス {index} は範囲外です。")
        return

    example = unmatched_data[index]
    current_data_id = example['id'] # 現在のIDを更新

    # generation_info をパース
    gen_info = parse_generation_info(example.get('generation_info', '{}'))
    answer_original_gi = gen_info.get('answer_original', 'N/A')
    answer_updated_gi = gen_info.get('answer_updated', 'N/A')
    error_info_gi = gen_info.get('error_info', gen_info.get('error', 'N/A')) # パースエラーも表示

    # --- データ表示エリアの更新 ---
    with output_display:
        clear_output(wait=True)
        display(Markdown(f"### データ {index + 1}/{num_unmatched} (ID: {example['id']})"))
        display(Markdown(f"**Question:**\n```\n{example['question']}\n```"))
        if example['choices']:
             display(Markdown(f"**Choices:** `{example['choices']}`"))

        # 回答比較表示
        compare_html = f"""
        <h4>回答比較:</h4>
        <table border="1" style="width:100%; border-collapse: collapse;">
          <tr>
            <th style="width:30%; padding: 5px;">項目</th>
            <th style="padding: 5px;">内容</th>
          </tr>
          <tr>
            <td style="padding: 5px;"><b>元の正解 (Dataset)</b></td>
            <td style="padding: 5px;"><code>{example['answer']}</code></td>
          </tr>
          <tr>
            <td style="padding: 5px;">元の正解 (gen_info)</td>
            <td style="padding: 5px;"><code>{answer_original_gi}</code></td>
          </tr>
          <tr>
            <td style="padding: 5px;">モデル生成回答 (gen_info)</td>
            <td style="padding: 5px;"><code>{answer_updated_gi}</code></td>
          </tr>
           <tr>
            <td style="padding: 5px;">エラー情報 (gen_info)</td>
            <td style="padding: 5px;"><span style="color: red;">{error_info_gi}</span></td>
          </tr>
        </table>
        <hr>
        """
        display(widgets.HTML(compare_html))

        display(Markdown(f"**CoT (Chain of Thought):**\n```\n{example['cot']}\n```"))
        display(Markdown(f"**Explanation:**\n```\n{example['explanation']}\n```"))

    # --- 編集エリアの更新 ---
    with output_edit:
        clear_output(wait=True)
        # 現在のデータを編集ウィジェットにセット
        data_to_edit = corrected_data_dict.get(current_data_id, example)

        answer_input.value = str(data_to_edit['answer'])
        cot_input.value = data_to_edit['cot']
        explanation_input.value = data_to_edit['explanation']

        # 修正状態を表示
        if current_data_id in corrected_data_dict:
             status_label.value = f"<b style='color:green;'>ID: {current_data_id} は修正済みです</b>"
        else:
             status_label.value = f"<span style='color:orange;'>ID: {current_data_id} は未修正です</span>"

        display(widgets.VBox([
            status_label,
            answer_input,
            cot_input,
            explanation_input,
            widgets.HBox([save_button, skip_button])
        ]))

    # ナビゲーションボタンの状態更新
    prev_button.disabled = (index == 0)
    next_button.disabled = (index == num_unmatched - 1)
    index_label.value = f"データ: {index + 1} / {num_unmatched} (ID: {current_data_id})"
    progress_bar.value = index + 1


def update_data_list():
    """全データリストを更新して表示"""
    with list_output:
        clear_output(wait=True)
        html_content = "<ul>"
        for idx, item in enumerate(unmatched_data):
            item_id = item['id']
            status_icon = "✅" if item_id in corrected_data_dict else "❌"
            status_text = "修正済み" if item_id in corrected_data_dict else "未修正"
            # 現在表示中の項目を太字にする
            style = "font-weight: bold;" if idx == current_index else ""
            html_content += f"<li style='{style}'>{status_icon} ID: {item_id} ({status_text})</li>"
        html_content += "</ul>"
        display(widgets.HTML(html_content))

def on_accordion_change(change):
    """アコーディオンが開かれたときにリストを更新"""
    if change['new'] == 0: # 0番目のタブ（リスト）が開かれた場合
        update_data_list()

def on_prev_button_clicked(b):
    """前のデータへ移動"""
    global current_index
    if current_index > 0:
        current_index -= 1
        display_data(current_index)
        # リストが表示されていれば更新
        if accordion.selected_index == 0:
            update_data_list()


def on_next_button_clicked(b):
    """次のデータへ移動"""
    global current_index
    if current_index < num_unmatched - 1:
        current_index += 1
        display_data(current_index)
         # リストが表示されていれば更新
        if accordion.selected_index == 0:
            update_data_list()

def on_save_button_clicked(b):
    """修正内容を保存し、次へ進む"""
    global corrected_data_dict
    target_id = current_data_id
    if not target_id: return

    # 元のデータをコピーして編集内容で上書き
    corrected_example = unmatched_data[current_index].copy()

    # Answer の処理
    try:
        new_answer = ast.literal_eval(answer_input.value)
        if not isinstance(new_answer, list):
            raise ValueError("入力はリスト形式である必要があります。例: ['a', 'b']")
        corrected_example['answer'] = new_answer
    except Exception as e:
        status_label.value = f"<b style='color:red;'>エラー: Answerの形式が不正です。 {e}</b>"
        return # 保存処理を中断

    # CoT, Explanation の更新
    corrected_example['cot'] = cot_input.value
    corrected_example['explanation'] = explanation_input.value

    # 修正済み辞書に保存
    corrected_data_dict[target_id] = corrected_example
    status_label.value = f"<b style='color:green;'>ID: {target_id} の修正を保存しました</b>"

    # リストが表示されていれば更新
    if accordion.selected_index == 0:
            update_data_list()

    # 少し待ってから次に進む (ステータス表示を見せるため、任意)
    time.sleep(0.5)

    # 次のデータへ（最後のデータでなければ）
    if current_index < num_unmatched - 1:
        on_next_button_clicked(None)
    else:
        # 最後のデータの場合はメッセージを表示
        status_label.value += " (最後のデータです)"


def on_skip_button_clicked(b):
    """現在のデータをスキップして次へ"""
    global corrected_data_dict
    target_id = current_data_id
    # スキップする場合、もし修正済みリストにあれば削除する
    if target_id in corrected_data_dict:
        del corrected_data_dict[target_id]
        status_label.value = f"<span style='color:orange;'>ID: {target_id} の修正を取り消しました</span>"
        # リストが表示されていれば更新
        if accordion.selected_index == 0:
            update_data_list()
        time.sleep(0.5) # メッセージ表示のため少し待つ
    else:
        status_label.value = ""

    # 次のデータへ
    if current_index < num_unmatched - 1:
        on_next_button_clicked(None)
    else:
        # 最後のデータの場合はメッセージを表示
         status_label.value = "スキップしました (最後のデータです)"


def on_export_button_clicked(b):
    """修正済みデータをJSON Lines形式でエクスポート"""
    with export_output:
        clear_output()
        if not corrected_data_dict:
            print("エクスポートする修正済みデータがありません。")
            return

        output_filename = f"corrected_unmatched_data_{len(corrected_data_dict)}items.jsonl"
        try:
            with open(output_filename, 'w', encoding='utf-8') as f:
                # ID順にソートしてエクスポートすると一貫性が保たれる（任意）
                sorted_ids = sorted(corrected_data_dict.keys())
                for item_id in sorted_ids:
                    item_data = corrected_data_dict[item_id]
                    f.write(json.dumps(item_data, ensure_ascii=False) + '\n')
            print(f"修正済みデータ {len(corrected_data_dict)} 件を '{output_filename}' に保存しました。")
            # Colab環境でファイルをダウンロードするリンクを表示
            try:
                from google.colab import files
                files.download(output_filename)
            except ImportError:
                 print("Google Colab環境外のため、自動ダウンロードはできません。")

        except Exception as e:
            print(f"エクスポート中にエラーが発生しました: {e}")

# --- イベントハンドラの登録 ---
prev_button.on_click(on_prev_button_clicked)
next_button.on_click(on_next_button_clicked)
save_button.on_click(on_save_button_clicked)
skip_button.on_click(on_skip_button_clicked)
export_button.on_click(on_export_button_clicked)
accordion.observe(on_accordion_change, names='selected_index') # アコーディオンの開閉イベント

# --- UIレイアウト ---
# 上部: プログレスバー
# 中部: データ表示、編集エリア、全データリスト(アコーディオン)
# 下部: エクスポート、ナビゲーションバー
ui = widgets.VBox([
    progress_bar,
    output_display, # データ表示エリア
    output_edit,    # 編集エリア
    accordion,      # 全データリスト表示エリア
    widgets.HTML("<hr>"), # 区切り線
    export_button,  # エクスポートボタン
    export_output,  # エクスポート結果表示エリア
    nav_bar         # ナビゲーションバーを一番下に移動
])

# --- 初期表示 ---
display(ui)
display_data(current_index) # 最初のデータを表示
# update_data_list() # 初期表示時にリストを読み込む場合はコメント解除

In [24]:
corrected_data_dict['115C41']['question'] = '25歳の女性。数か月前に虫垂炎で入院した際行われた腹部超音波検査で腎臓の異常を指摘され、母の腎臓病が遺伝していないか心配で検査を希望して来院した。母は58歳で、遺伝性腎疾患のため数か月前から透析をしている。母方祖父も60歳から同病で透析をしており、数年前に脳出血で亡くなった。父方の家系に同病の人はいない。身長160cm、体重51kg。血圧110/70 mmHg。脈拍80/分、整。心音と呼吸音とに異常を認めない。腹部は平坦、軟で、腫瘤は触知しない。尿所見：蛋白(-)、糖(-)、潜血反応(-)。腹部造影CT 別冊No. 8を別に示す。母と同病であると診断された。近く、結婚予定で挙児希望がある。パートナーの家系に同病の人はいない。この患者の子どもが同遺伝性腎疾患を有する確率はどれか。男児 % 女児 %'
corrected_data_dict['115C41']['choices'] = [
    'a. 0, 0',
    'b. 50, 0',
    'c. 0, 50',
    'd. 25, 25',
    'e. 50, 50'
]
corrected_data_dict['110G11']['choices'] = [
    'a. T0 / l0',
    'b. Tx / lx',
    'c. T(x+1) / l(x+1)',
    'd. T0 / l0 - x',
    'e. Tx / lx + x'
]

## Upload to Hugging Face

In [31]:
import datasets
from datasets import Dataset, DatasetDict, concatenate_datasets, Features, Value
from huggingface_hub import HfApi, create_repo, HfFolder

# --- ステップ 1: Hugging Face Hub へのログイン ---
print("Hugging Face Hub にログインしてください。")
print("アクセストークンが必要です (write権限)。 https://huggingface.co/settings/tokens で作成できます。")
# from huggingface_hub import notebook_login
# notebook_login()

# --- ステップ 2: 必要なデータの確認 ---
# (前のセルと同じコードなので省略)
proceed = True # 仮にTrueとする。実際は前のセルのチェックを入れる
if 'dataset' not in globals():
    print("\n❌ エラー: 元の 'dataset' オブジェクトが見つかりません。")
    proceed = False
# ... (corrected_data_dict のチェックも同様) ...
if proceed and 'corrected_data_dict' not in globals():
     print("\n❌ エラー: 'corrected_data_dict' が見つかりません。")
     proceed = False
# ... (corrected_data_dict が空の場合の処理も同様) ...
elif proceed and not corrected_data_dict:
     print("\n⚠️ 警告: 修正済みのデータがありません ('corrected_data_dict' が空です)。")
     # ... (続行確認など) ...
     corrected_unmatched_ds_prepared = None # 空の場合の準備
else:
     print(f"\n✅ 修正済みのデータが {len(corrected_data_dict)} 件見つかりました。")


# --- ステップ 3: データセットの準備と由来情報の追加 ---
original_train_ds_prepared = None
corrected_unmatched_ds_prepared = None

if proceed:
    # 3a: 元の 'train' スプリットに 'source' カラムを追加
    try:
        original_train_ds = dataset['train']
        # .add_column() を使って新しいカラムを追加
        original_train_ds_prepared = original_train_ds.add_column(
            name="source", # カラム名
            column=["matched"] * len(original_train_ds) # 全ての行に 'matched' を設定
        )
        print("\n元の 'train' スプリットに 'source' カラム ('matched') を追加しました:")
        print(original_train_ds_prepared)
        print("新しいFeatures:", original_train_ds_prepared.features) # スキーマ確認
    except KeyError:
        print("\n❌ エラー: 元のデータセットに 'train' スプリットが見つかりません。")
        proceed = False
    except Exception as e:
        print(f"\n❌ エラー: 元の 'train' データへのカラム追加中にエラー: {e}")
        proceed = False

if proceed and corrected_data_dict: # 修正済みデータがある場合のみ処理
    # 3b: 修正済みデータに 'source' カラムを追加して Dataset オブジェクトに変換
    try:
        corrected_list = list(corrected_data_dict.values())
        # 各辞書に 'source': 'unmatched' を追加
        for item in corrected_list:
            item['source'] = 'unmatched'

        # 元のスキーマを取得し、'source' カラム定義を追加
        # (original_train_ds_prepared が正常に作成されている前提)
        if original_train_ds_prepared:
            new_features_dict = original_train_ds_prepared.features.copy()
            # 既存のスキーマに source がなければ追加 (add_columnで追加済みなら不要だが念のため)
            if 'source' not in new_features_dict:
                 new_features_dict['source'] = Value('string')
            new_features = Features(new_features_dict)

            corrected_unmatched_ds_prepared = Dataset.from_list(
                corrected_list,
                features=new_features # sourceカラムを含むスキーマを指定
            )
            print("\n修正済み 'unmatched' データに 'source' カラム ('unmatched') を追加し、Datasetに変換しました:")
            print(corrected_unmatched_ds_prepared)
            print("Features:", corrected_unmatched_ds_prepared.features) # スキーマ確認
        else:
            print("\n❌ エラー: スキーマの基準となる元のTrainデータ準備に失敗したため、Unmatchedデータを処理できません。")
            proceed = False

    except Exception as e:
        print(f"\n❌ エラー: 修正済みデータの Dataset への変換中に問題が発生しました: {e}")
        print("データの形式やスキーマを確認してください。処理を中断します。")
        proceed = False

# --- ステップ 4: データセットの結合 ---
new_train_ds = None # 初期化

if proceed:
    print("\n'source' カラム付きのデータセットを結合して新しい 'train' スプリットを作成します...")
    datasets_to_combine = []
    if original_train_ds_prepared:
        datasets_to_combine.append(original_train_ds_prepared)
    # corrected_unmatched_ds_prepared が None でない（＝修正データがあった）場合のみ追加
    if corrected_unmatched_ds_prepared:
        datasets_to_combine.append(corrected_unmatched_ds_prepared)

    if not datasets_to_combine:
        print("\n❌ エラー: 結合するデータセットがありません。")
        proceed = False
    else:
        try:
            # スキーマが一致していることを確認 (任意だが推奨)
            if len(datasets_to_combine) > 1:
                features1 = datasets_to_combine[0].features
                features2 = datasets_to_combine[1].features
                if features1.keys() != features2.keys():
                     print("\n⚠️ 警告: 結合するデータセットのスキーマ（カラム名）が完全に一致しません。")
                     print(f"  スキーマ1: {list(features1.keys())}")
                     print(f"  スキーマ2: {list(features2.keys())}")
                     # 必要ならここでエラーにするか、続行するか判断
                # else:
                #    print("\n✅ 結合するデータセットのスキーマは一致しています。")

            if len(datasets_to_combine) == 1:
                 new_train_ds = datasets_to_combine[0]
                 print("\nデータセットが1種類のみのため、結合はスキップされました。")
            else:
                 new_train_ds = concatenate_datasets(datasets_to_combine)
                 print("\n新しい 'train' スプリットを作成しました (sourceカラム付き):")

            print(new_train_ds) # 結合後のデータセット情報を表示
            print(f"サンプルデータ (sourceカラム確認): {new_train_ds[0]}")
            # データセットの最後の方も確認（unmatchedデータが来ているか）
            if len(new_train_ds) > len(original_train_ds):
                 print(f"サンプルデータ2 (sourceカラム確認): {new_train_ds[len(original_train_ds)]}")


        except Exception as e:
            print(f"\n❌ データセットの結合中にエラーが発生しました: {e}")
            print("各データセットのスキーマ(features)が一致しているか確認してください。")
            proceed = False

# --- ステップ 5: 最終的な DatasetDict を作成 ---
# (前のセルと同じコード)
final_upload_dict = None # 初期化
if proceed and new_train_ds:
    final_upload_dict = DatasetDict({'train': new_train_ds})
    print("\n最終的なアップロード用 DatasetDict を作成しました ('train' スプリットのみ, sourceカラム付き):")
    print(final_upload_dict)
else:
     print("\n最終的な DatasetDict を作成できませんでした。")
     proceed = False

# --- ステップ 6 & 7: アップロード先指定と実行 ---
# (前のセルと同じコードなので省略)
if proceed:
    # ... (リポジトリID入力、プライベート設定) ...
    try:
         hf_api = HfApi()
         user_info = hf_api.whoami()
         hf_username = user_info['name']
         original_repo_name = dataset.builder_name
         if '/' in original_repo_name:
             original_repo_name = original_repo_name.split('/')[-1]
         default_repo_id = f"{hf_username}/{original_repo_name}-combined-sourced" # サフィックス変更
         print(f"\nあなたの Hugging Face ユーザー名: {hf_username}")
    except Exception:
         hf_username = "your-username"
         default_repo_id = f"{hf_username}/JMLE-CoT-gemini-2.5-pro-dataset-combined-sourced"
         print("\n⚠️ Hugging Face ユーザー名の自動取得に失敗しました。")

    repo_id = input(f"アップロード先の新しいリポジトリIDを入力してください (例: {default_repo_id}): ")
    # ... (入力処理、プライベート設定) ...
    if not repo_id.strip(): repo_id = default_repo_id
    if '/' not in repo_id: repo_id = f"{hf_username}/{repo_id}"
    is_private_str = input("データセットをプライベートにしますか？ (yes/no, デフォルト: yes): ").lower().strip()
    is_private = is_private_str != 'no'

    try:
        print(f"\n'{repo_id}' に結合済みデータセット (sourceカラム付き) をアップロードします (Private: {is_private})...")
        final_upload_dict.push_to_hub(repo_id=repo_id, private=is_private)
        print("\n✅ アップロードが完了しました！")
        # ... (URL表示など) ...
        hub_url = f"https://huggingface.co/datasets/{repo_id}"
        print(f"データセットはこちらで確認できます: {hub_url}")

    except Exception as e:
        print(f"\n❌ アップロード中にエラーが発生しました: {e}")


# 処理が中断された場合のメッセージ
if not proceed:
    print("\n処理が途中で中断されたため、アップロードは行われませんでした。")

Hugging Face Hub にログインしてください。
アクセストークンが必要です (write権限)。 https://huggingface.co/settings/tokens で作成できます。

✅ 修正済みのデータが 34 件見つかりました。

元の 'train' スプリットに 'source' カラム ('matched') を追加しました:
Dataset({
    features: ['id', 'question', 'choices', 'answer', 'cot', 'explanation', 'generation_info', 'source'],
    num_rows: 3356
})
新しいFeatures: {'id': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'choices': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'answer': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'cot': Value(dtype='string', id=None), 'explanation': Value(dtype='string', id=None), 'generation_info': Value(dtype='string', id=None), 'source': Value(dtype='string', id=None)}

修正済み 'unmatched' データに 'source' カラム ('unmatched') を追加し、Datasetに変換しました:
Dataset({
    features: ['id', 'question', 'choices', 'answer', 'cot', 'explanation', 'generation_info', 'source'],
    num_rows: 34
})
Features: {'id': Value(dtype='string'

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]


✅ アップロードが完了しました！
データセットはこちらで確認できます: https://huggingface.co/datasets/doctorin/JMLE-CoT-gemini-2.5-pro-dataset-combined


## View Output

In [33]:
dataset_corrected = display_dataset_summary("doctorin/JMLE-CoT-gemini-2.5-pro-dataset-combined")

README.md:   0%|          | 0.00/539 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.69M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3390 [00:00<?, ? examples/s]

Dataset Structure:
DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'choices', 'answer', 'cot', 'explanation', 'generation_info', 'source'],
        num_rows: 3390
    })
})

Split: train
Number of examples: 3390

Columns:
  - id: Value(dtype='string', id=None)
  - question: Value(dtype='string', id=None)
  - choices: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
  - answer: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
  - cot: Value(dtype='string', id=None)
  - explanation: Value(dtype='string', id=None)
  - generation_info: Value(dtype='string', id=None)
  - source: Value(dtype='string', id=None)

Sample data (first 3 rows):

Sample 1:
  id: 117A01
  question: 母体背景と胎児疾患の組合せで正しいのはどれか。3 つ選べ。 
  choices: ['a. 高齢妊娠 ― 13trisomy', 'b. 風疹感染 ― 先天性心疾患', 'c. 妊娠高血圧症候群 ― 不整脈', 'd. 全身性エリテマトーデス ― 頭蓋内出血', 'e. パルボウイルス B19 感染 ― 貧血']
  answer: ['a', 'b', 'e']
  cot: 1.  問題文を分析する。母体の背景因子（年齢、感染症、基礎疾患）と、それが引き起こす可能性のある胎児疾患の正しい組み合わせを3つ選ぶ